Дополнительный проект
В папке subsid (shared/homeworks/python_ds_miniprojects/5_subsid) находятся файлы (tm_sales_1, tm_sales_2, ...) с продажами продуктов через телемаркетинг. Каждый файл содержит, как минимум, 4 колонки (поля): FILIAL_ID, SUBS_ID, PROD_ID, ACT_DTTM.

Суть задачи в том, чтобы проверить подключения продуктов определенным пользователям, соединив файлы о продажах с логами по подключениям в системе.

Особенности данных:

сотрудники телемаркетинга не всегда указывают полный id, если 'id' нет в начале SUBS_ID, то нужно его добавить
поля в файлах могут быть расположены абсолютно случайным образом, но названия полей статичны
продажа не засчитывается, если отключение (END_DTTM) произошло меньше чем через 5 минут после подключения (START_DTTM)
если в файле с продажами встречается строка без указанного SUBS_ID, она пропускается
Сохраните результат в датафрэйм с разделителем ;, содержащий корректные подключения.

Note: обратите внимание на то, как pandas переводит дату из строки, возможно вам понадобится параметр format
Пример содержания итогового файла (колонка difference это разница между START_DTTM и END_DTTM):

ACT_DTTM;FILIAL_ID;difference
20-03-2020 14:59;1;255 days 09:01:00

In [185]:
import pandas as pd
import os

path = "/home/jupyter-v-scherbak-20/shared/homeworks/python_ds_miniprojects/5_subsid/subsid/"
#dirs = os.listdir(path)
#l = len(dirs)
#for i in dirs:
#    a = pd.read_csv(path + i, sep=";")

In [186]:
tm_sales_1 = pd.read_csv(path + "tm_sales_1.csv", sep=";")

In [187]:
tm_sales_2 = pd.read_csv(path + "tm_sales_2.csv", sep=";")

In [188]:
tm_sales_3 = pd.read_csv(path + "tm_sales_3.csv", sep=";")

In [189]:
prod_activations_logs = pd.read_csv(path + "prod_activations_logs.csv", sep=";")
prod_activations_logs.START_DTTM = pd.to_datetime(prod_activations_logs.START_DTTM, format='%d-%m-%Y %H:%M')
prod_activations_logs.END_DTTM = pd.to_datetime(prod_activations_logs.END_DTTM, format='%d-%m-%Y %H:%M')

In [190]:
df = pd.concat([tm_sales_1, tm_sales_2, tm_sales_3])

In [191]:
def add_id(str_id):
    if not str_id.startswith('id'):
        str_id = "id" + str_id
        return str_id
    else:
        return str_id

In [192]:
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
0,id5416547,3,1743,17-03-2020 10:17
1,NaN,4,7421,12-03-2020 11:25
2,8362218,7,9879,05-03-2020 11:42
3,2185490,2,3210,16-03-2020 16:28
0,id5764122,3,1499,18-03-2020 15:44
1,id7642700,6,3020,15-03-2020 14:21
2,id1374509,2,5677,17-03-2020 11:48


In [193]:
df = df.dropna() 

In [194]:
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
0,id5416547,3,1743,17-03-2020 10:17
2,8362218,7,9879,05-03-2020 11:42
3,2185490,2,3210,16-03-2020 16:28
0,id5764122,3,1499,18-03-2020 15:44
1,id7642700,6,3020,15-03-2020 14:21
2,id1374509,2,5677,17-03-2020 11:48


In [195]:
df.SUBS_ID = df.SUBS_ID.apply(add_id)

/opt/tljh/user/lib/python3.7/site-packages/pandas/core/generic.py:5165: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value


In [196]:
df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM
0,id4651830,1,1954,20-03-2020 14:59
1,id7646509,5,6431,19-03-2020 13:00
2,id7412683,4,3313,22-03-2020 17:25
0,id5416547,3,1743,17-03-2020 10:17
2,id8362218,7,9879,05-03-2020 11:42
3,id2185490,2,3210,16-03-2020 16:28
0,id5764122,3,1499,18-03-2020 15:44
1,id7642700,6,3020,15-03-2020 14:21
2,id1374509,2,5677,17-03-2020 11:48


In [197]:
full_df = df.merge(prod_activations_logs, on=("SUBS_ID", "PROD_ID"))
#full_df.dtypes

In [198]:
full_df["time"] = full_df.END_DTTM - full_df.START_DTTM

In [199]:
full_df

,SUBS_ID,FILIAL_ID,PROD_ID,ACT_DTTM,START_DTTM,END_DTTM,time
0,id4651830,1,1954,20-03-2020 14:59,2020-03-20 14:59:00,2020-12-01 00:00:00,255 days 09:01:00
1,id7646509,5,6431,19-03-2020 13:00,2020-03-19 13:00:00,2020-03-19 13:03:00,0 days 00:03:00
2,id5416547,3,1743,17-03-2020 10:17,2020-03-17 10:17:00,2020-03-25 11:00:00,8 days 00:43:00
3,id2185490,2,3210,16-03-2020 16:28,2020-03-16 16:28:00,2020-12-01 00:00:00,259 days 07:32:00
4,id7642700,6,3020,15-03-2020 14:21,2020-03-15 14:21:00,2020-03-15 23:42:00,0 days 09:21:00


In [213]:
full_df = full_df.query("time >'5m'")

In [214]:
print(full_df.sort_values("SUBS_ID").SUBS_ID.to_list())

['id2185490', 'id4651830', 'id5416547', 'id7642700']
